In [ ]:
# importing dependencies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.linalg import null_space


In [ ]:
# function to run the Randomized Kaczmarz algorithm
def RK(A, b, x_ref):
    " this function takes as inputs a matrix A, vector b, and a point x_star and returns "
    "the mean approximation error E[||x_k - x_ref||^2] and the mean of the iterates E[x_k] across multiple runs"

    # generating probabilities of choosing the rows
    frob_norm_A = np.linalg.norm(A, ord='fro')
    probas = (np.linalg.norm(A, axis=1)/frob_norm_A)**2

    approx_error = np.zeros((n_iter+1,))
    sum_iterates = np.zeros((n_iter+1,A.shape[1]))
       
    for r in range(int(n_run)):

        x = x_0   # initialization
        
        run_approx_error = []
        run_approx_error.append(np.linalg.norm(x - x_ref)**2)
        run_iterates = []
        run_iterates.append(x)

        for i in range(int(n_iter)):
            row_idx = int(np.random.choice(A.shape[0], 1, p=probas))
            if np.linalg.norm(A[row_idx,:])==0:
                continue
            else:
                x = x + (b[row_idx] - np.dot(A[row_idx,:],x))/((np.linalg.norm(A[row_idx,:], ord=2))**2)*A[row_idx,:]
                run_approx_error.append(np.linalg.norm(x - x_ref)**2)
                run_iterates.append(x)
        
        sum_iterates += np.array(run_iterates)
        approx_error += np.array(run_approx_error)
                
        print("end run", r)

    return(approx_error/(n_run),sum_iterates/(n_run))

In [ ]:
# Generating the noisy data
m = 1000
n = 500
r = 300
noisy_A = np.random.randn(m,r) @ np.random.randn(r,n)
basis_nullSpaceNoisyAT = null_space(noisy_A.T)
w = basis_nullSpaceNoisyAT@np.random.randn(basis_nullSpaceNoisyAT.shape[1],)  # w is a vector in the orthogonal complement of the column space of noisy_A
w = w/np.linalg.norm(w, ord=2)   # ||w||=1
y = noisy_A@np.random.randn(n,)  # y is a vector in the column space of noisy_A
beta = 10
noisy_b = y + beta*w
# noisy_b = np.zeros((m,))



# ####### Using Real-World Data
# data = pd.read_csv('a1a.csv', sep=',', header=None)
# print(data.head())
# m = data.shape[0]
# n = data.shape[1]-1
# noisy_A = np.array(data.iloc[: , 0:n])
# noisy_b = np.array(data.iloc[: , -1])

# rank_noisy_A = np.linalg.matrix_rank(noisy_A)
# augg = np.column_stack((noisy_A,noisy_b))
# rank_aug = np.linalg.matrix_rank(augg)
# print("noisy_A is full rank:", rank_noisy_A==n)
# print("rank(noisy_A) = ",rank_noisy_A)
# print("rank(noisy_A|noisy_b) = ",rank_aug)
# print("system is consistent:  ", rank_noisy_A == rank_aug)

In [ ]:
# starting point
x_0 = 40*np.random.randn(n,) 

In [ ]:
# reference point
# x_star = np.random.randn(n,)   # x_star is random
x_star = np.linalg.pinv(noisy_A)@noisy_b  # x_star = the least squares solution of the noisy system
# U, s, Vt = np.linalg.svd(noisy_A, full_matrices=True)
# x_star = Vt[r-1,:]  # x_star = the smallest right singular vector

In [ ]:
# projection of x_0 on the null space of noisy_A
null_spaceNoisyA = null_space(noisy_A)
orthogonal_basis = null_spaceNoisyA
dot_product_values = (x_0.T) @ orthogonal_basis
mult_basis = np.zeros(orthogonal_basis.shape)
for i in range(len(dot_product_values)):
    mult_basis[:,i] = dot_product_values[i]*orthogonal_basis[:,i]
x0_n =  np.sum( mult_basis , axis=1)
x0_r = x_0 - x0_n




# projection of x_star on the row space of noisy_A (i.e., the column space of noisy_A.T)
dot_product_values = (x_star.T) @ orthogonal_basis
mult_basis = np.zeros(orthogonal_basis.shape)
for i in range(len(dot_product_values)):
    mult_basis[:,i] = dot_product_values[i]*orthogonal_basis[:,i]
x_star_n =  np.sum( mult_basis , axis=1)
x_star_r = x_star - x_star_n

In [ ]:
# Running the RK algorithm

n_iter = 150000   # number of iterations
n_run = 20        # number of runs of the algorithm

approx_error, mean_iterates = RK(noisy_A, noisy_b, x0_n+x_star_r)

In [ ]:
# computing the approximation errors
approx_error_sqrt =  np.sqrt(approx_error)  # approx_error_sqrt = sqrt(E[||x_k-x_0^n-x_*^r||^2])
approx_error_norm = np.linalg.norm(mean_iterates-x0_n-x_star_r, axis=1) # approx_error_norm = ||E[x_k]-x_0^n-x_*^r||

In [ ]:
# computing the smallest non zero singular value of noisy_A
U, s, Vt = np.linalg.svd(noisy_A)
s = [x for x in s if x>1e-8]
sigma_min = min(s)

# computing the bounds
bound_thm = [] # square root of bound of Theorem 2.2
bound_cor = [] # bound of Corollary 3.5
Rtld = (np.linalg.norm(noisy_A, ord='fro')/sigma_min)**2
h = np.linalg.norm(noisy_A @ x_star - noisy_b, ord=2)/sigma_min  # horizon
for k in range(n_iter):
    bound_thm.append(sqrt((1-1/Rtld)**(k)*(np.linalg.norm(x0_r - x_star_r, ord=2)**2) + h**2))
    bound_cor.append((1-1/Rtld)**(k)*np.linalg.norm(x0_r - x_star_r, ord=2) + h)


In [ ]:
# plotting the results

plt.figure(figsize=(9.5, 5.5))

plt.plot(bound_thm, label='Bound Theorem 2.2', color='#e41a1c', linestyle='dashdot', linewidth=3)
plt.plot(bound_cor, label='Bound Corollary 3.5', color='#4daf4a', linestyle='dashed', linewidth=3)
plt.plot(approx_error_sqrt, label=r'$\sqrt{\mathbf{\mathbb{E}}\left[\Vert x_k-x_0^n- x_{*}^r \Vert ^2 \right]}$', color='#377eb8', linewidth=3, marker="^", markersize=10, markevery=20000)
plt.plot(approx_error_norm, label=r'$\Vert \mathbf{\mathbb{E}}\left[x_k\right]-x_0^n- x_{*}^r \Vert$', color = '#984EA3', linewidth=3, marker="x", markersize=15, markeredgewidth=2, markevery=23000)

plt.title(r'$m=%d$'%(m)+',  '+r'$n=%d$'%(n)+',  '+r'$rank(\widetilde A)=%d$'%(np.linalg.matrix_rank(noisy_A)), fontsize = 24)
plt.xlabel("Iterations",fontsize = 26)
plt.rcParams['xtick.labelsize']=24
plt.rcParams['ytick.labelsize']=24
plt.legend(fontsize=23)
plt.xlim(0,n_iter)
plt.yscale('log')
plt.grid(linestyle = '--')
plt.ticklabel_format(style='sci', axis='x', scilimits=(3,3))